In [10]:
import pandas as pd

games = pd.read_csv('../datasets/finals/final.csv')

games.drop(['Unnamed: 0'], axis=1, inplace=True)
games = games.fillna('')
games['release_date'] = pd.to_datetime(games['release_date']).dt.strftime("%Y-%m-%dT%H:%M:%SZ")

games.head()

,name,platform,release_date,summary,meta_score,user_review,MetacriticReviews,Wikipedia,Genre,Modes
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23T00:00:00Z,"As a young boy, Link is tricked by Ganondorf, ...",99,9.1,"[('Nintendorks', 'The biggest, most heartpound...",The Legend of Zelda: Ocarina of Time 3D is a 2...,Action-adventure,['Single-player']
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20T00:00:00Z,As most major publishers' development efforts ...,98,7.4,"[('CNET Gamecenter', 'The best skateboarding g...",Tony Hawk's Pro Skater 2 is a skateboarding vi...,Sports,"['Single-player', 'multiplayer']"
2,Grand Theft Auto IV,PlayStation 3,2008-04-29T00:00:00Z,[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7,"[('Playstation Official Magazine UK', ""GTA IV ...",Grand Theft Auto IV is a 2008 action-adventure...,Action-adventure,"['Single-player', 'multiplayer']"
3,SoulCalibur,Dreamcast,1999-09-08T00:00:00Z,"This is a tale of souls and swords, transcendi...",98,8.4,"[('CheckOut', ""I guarantee you'll be amazed at...",Soulcalibur is a weapon-based 3D fighting game...,Fighting,"['Single-player', 'multiplayer']"
4,Grand Theft Auto IV,Xbox 360,2008-04-29T00:00:00Z,[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9,"[('Official Xbox Magazine UK', ""It's got a wor...",Grand Theft Auto IV is a 2008 action-adventure...,Action-adventure,"['Single-player', 'multiplayer']"


In [11]:
from ast import literal_eval

# In some entries the user review is marked as "tbd", this isn't compatible with Solr's FloatPointField type so it should be stored as "NaN" instead
def get_user_review(user_review):
    if user_review == "tbd":
        return "NaN"
    return user_review

def get_platform_reviews_json(game):
    platforms = list(games[games.name == game]["platform"])
    json = []

    for platform in platforms:
        reviews = (games[(games.name == game) & (games.platform == platform)]).iloc[0].MetacriticReviews
        for review in eval(reviews):
            json.append({
                "platform": (games[(games.name == game) & (games.platform == platform)]).iloc[0].platform,
                "release_date": (games[(games.name == game) & (games.platform == platform)]).iloc[0].release_date,
                "meta_score": str((games[(games.name == game) & (games.platform == platform)]).iloc[0].meta_score),
                "user_review": get_user_review((games[(games.name == game) & (games.platform == platform)]).iloc[0].user_review),
                "reviewer": review[0],
                "review": review[1]
            })
    return json

def get_game_json(game):
    json = {
        "name": game,
        "summary": (games[games.name == game]).iloc[0].summary,
        "wikipedia": (games[games.name == game]).iloc[0].Wikipedia,
        "genre": (games[games.name == game]).iloc[0].Genre,
        "playing_modes": literal_eval((games[games.name == game]).iloc[0].Modes) if (games[games.name == game]).iloc[0].Modes else [],
        "content_type": "parentDocument",
        "reviews": get_platform_reviews_json(game)
    }
    return json

In [13]:
from json import dumps

json = []
for game in games['name'].unique():
    print(game)
    json.append(get_game_json(game))

json = dumps(json, indent=4)
with open("../search-engine/games_collection.json", "w") as outfile:
    outfile.write(json)

The Legend of Zelda: Ocarina of Time
Tony Hawk's Pro Skater 2
Grand Theft Auto IV
SoulCalibur
Super Mario Galaxy
Super Mario Galaxy 2
Red Dead Redemption 2
Grand Theft Auto V
Disco Elysium: The Final Cut
The Legend of Zelda: Breath of the Wild
Tony Hawk's Pro Skater 3
Perfect Dark
Metroid Prime
Grand Theft Auto III
Super Mario Odyssey
Halo: Combat Evolved
NFL 2K1
BioShock
GoldenEye 007
Uncharted 2: Among Thieves
The Orange Box
Batman: Arkham City
Tekken 3
Mass Effect 2
The Legend of Zelda: Twilight Princess
The Elder Scrolls V: Skyrim
The Legend of Zelda: The Wind Waker
Gran Turismo
Metal Gear Solid 2: Sons of Liberty
Grand Theft Auto Double Pack
Baldur's Gate II: Shadows of Amn
Grand Theft Auto: San Andreas
Grand Theft Auto: Vice City
LittleBigPlanet
The Legend of Zelda Collector's Edition
Red Dead Redemption
Halo 2
The Legend of Zelda: A Link to the Past
The Legend of Zelda: Majora's Mask
The Last of Us
Madden NFL 2003
Persona 5 Royal
The Last of Us Remastered
Tetris Effect: Connecte